### Wrangling with missing or bogus data or any 0 values
- Either drop NaN data or fillin with reasonable value
- clean size information
- clean branding informaiton
- Calculate the wholesale price

### Import libraries and define universal variables

In [1]:
import numpy as np
import pandas as pd
import re
import math
import matplotlib.pyplot as plt
import codecs
from lib import utility as util
from functools import partial

### Import data
*** please don't proceec to the next cell before data tables is shown below the cell.***

In [2]:
# file paths
integrated_file = "processed_data/integrated.csv"
integrated_df = pd.read_csv( integrated_file, 
                             index_col='sales_date',
                             header=0,
                             parse_dates=['sales_date'],
                             low_memory=False )

sales_variables = ['retail_price','sales_proceeds','raw_cost','gross_profit','number_of_sales']

integrated_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876339 entries, 2014-08-01 to 2017-07-28
Data columns (total 20 columns):
product_id             876339 non-null object
sales_id               876339 non-null int64
product_name           876339 non-null object
retail_price           876339 non-null int64
sales_proceeds         876339 non-null object
gross_profit           873639 non-null object
number_of_sales        876339 non-null int64
raw_cost               876339 non-null float64
manufacture_country    856216 non-null object
category               875465 non-null object
material_type          875160 non-null object
size                   844820 non-null object
quality                836162 non-null object
brand                  607592 non-null object
color                  594584 non-null object
subcategory_group      876339 non-null object
subcategory            876339 non-null object
size_letter            209937 non-null object
material_group         876339 non-null object
m

### Fixing data types

In [3]:
# convert all string value, which should be of float type, to float type
integrated_df[sales_variables] = integrated_df[sales_variables].applymap(util.convert_to_float)

# drop rows with invalid numeric data 
print("The dataframe length before invalid numerical data is dropped: {}".format(len(integrated_df)))
integrated_df = util.filter_by_invalid_numerics_in_df(integrated_df, sales_variables)
print("The dataframe length after invalid numerical data is dropped: {}".format(len(integrated_df)))

# convert data to originally intended data types
integrated_df[sales_variables] = integrated_df[sales_variables].astype(float)
integrated_df['number_of_sales'] = integrated_df['number_of_sales'].astype(int)
integrated_df['sales_id'] = integrated_df['sales_id'].astype(object)
print(integrated_df.info())

The dataframe length before invalid numerical data is dropped: 876339
The dataframe length after invalid numerical data is dropped: 876337
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876337 entries, 2014-08-01 to 2017-07-28
Data columns (total 20 columns):
product_id             876337 non-null object
sales_id               876337 non-null object
product_name           876337 non-null object
retail_price           876337 non-null float64
sales_proceeds         876337 non-null float64
gross_profit           873637 non-null float64
number_of_sales        876337 non-null int64
raw_cost               876337 non-null float64
manufacture_country    856214 non-null object
category               875463 non-null object
material_type          875158 non-null object
size                   844818 non-null object
quality                836160 non-null object
brand                  607592 non-null object
color                  594582 non-null object
subcategory_group      876337 non-null ob

### Calculating gross profit and sold price

In [10]:
# possible whole sale price = (gross profit - ( raw_cost * # of sales)) / # of sales 
missing_gross_profits = integrated_df[integrated_df['gross_profit'].isnull()]
mraw_cost = missing_gross_profits['raw_cost'].apply(np.floor)
msales_proceeds = missing_gross_profits['sales_proceeds'].apply(np.floor)
mnumber_of_sales = missing_gross_profits['number_of_sales']
integrated_df.loc[integrated_df['gross_profit'].isnull(), 'gross_profits'] = msales_proceeds - (mraw_cost * mnumber_of_sales)

gross_profit = integrated_df['gross_profit'].apply(np.floor)
raw_cost = integrated_df['raw_cost'].apply(np.floor)
number_of_sales = integrated_df['number_of_sales']
integrated_df['sold_price'] = (gross_profit + (raw_cost * number_of_sales)) / number_of_sales
sales_variables.append('sold_price')
print(integrated_df)



           product_id  sales_id  \
sales_date                        
2014-08-01     103591  20239746   
2014-08-01     102862  20239747   
2014-08-01     104782  20239747   
2014-08-01     999000  20239747   
2014-08-01     190038  20239751   
2014-08-01     190039  20239751   
2014-08-01     190044  20239751   
2014-08-01     190046  20239751   
2014-08-01     190047  20239751   
2014-08-01     190064  20239751   
2014-08-01     190065  20239751   
2014-08-01     190066  20239751   
2014-08-01     190067  20239751   
2014-08-01   190094OR  20239751   
2014-08-01   190094WH  20239751   
2014-08-01     190117  20239751   
2014-08-01     190119  20239751   
2014-08-01   308353BL  20239751   
2014-08-01   308353GN  20239751   
2014-08-01   308353MT  20239751   
2014-08-01   308353NV  20239751   
2014-08-01   308353PK  20239751   
2014-08-01     371337  20239751   
2014-08-01     371338  20239751   
2014-08-01     371424  20239751   
2014-08-01     371426  20239751   
2014-08-01     19005

### Categorize brands

### Categorize sizes